In [ ]:
import os
import time
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import csv
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from featout_exp.utils import load_data, load_model

In [ ]:
BATCHSIZE = 32
NUM_EPOCHS = 10
EXP_NAME = "TEST"
TESTING = True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = load_model(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

train_loader = load_data("train", 32, True)
test_loader = load_data("test", 32, False)
EXP_PATH = os.path.join("logs",EXP_NAME)

os.makedirs(EXP_PATH, exist_ok=True)
# Initialize TensorBoard writer
writer = SummaryWriter()

# Prepare a CSV file for logging
csv_file = open(os.path.join(EXP_PATH, "logs.csv"), "w")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Epoch", "Train Loss", "Train Accuracy", "Test Loss", "Test Accuracy"])

In [ ]:
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    corrects = 0
    total = 0

    bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")

    for step, (inputs, labels) in enumerate(bar):
        
        if TESTING == True and step > 2:
            break
        
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = corrects.double() / total

    # Evaluate on test data
    model.eval()
    test_running_loss = 0.0
    test_corrects = 0
    test_total = 0

    with torch.no_grad():
        for step,  (inputs, labels) in enumerate(tqdm( test_loader)):
            if TESTING == True and step > 2:
                 break
             
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_running_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)
            test_total += labels.size(0)

    test_loss = test_running_loss / test_total
    test_acc = test_corrects.double() / test_total

    # Log metrics to TensorBoard
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Accuracy/train", train_acc, epoch)
    writer.add_scalar("Loss/test", test_loss, epoch)
    writer.add_scalar("Accuracy/test", test_acc, epoch)

    # Log metrics to CSV
    csv_writer.writerow([epoch + 1, train_loss, train_acc.item(), test_loss, test_acc.item()])
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} - Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}")

csv_file.close()
writer.close()


In [ ]:
torch.save(model.state_dict(), os.path.join(EXP_PATH, "model.pth"))